In [28]:
from google import genai
from pydantic import BaseModel
import json
import asyncio
from tqdm.asyncio import tqdm
from collections import defaultdict
import pandas as pd
from typing import List, Dict
from dotenv import load_dotenv
import os
from dotenv import load_dotenv

In [29]:

load_dotenv()
api_key = os.getenv("GOOGLE_GENAI_API_KEY")


In [16]:
data_path = "/home/vinh/HS Code/Data/df.csv"

In [17]:
# Đọc file CSV, ép kiểu tất cả các cột thành chuỗi
df = pd.read_csv(data_path, dtype=str)
data = df.head(12)
data = data[['mahs', 'mo_ta']]

In [18]:
data

,mahs,mo_ta
0,01012900,"Ngựa, lừa, la sống/- Ngựa:/- - Loại khác"
1,01013010,"Ngựa, lừa, la sống/- Lừa:/- - Loại thuần chủng..."
2,01013090,"Ngựa, lừa, la sống/- Lừa:/- - Loại khác"
3,01019000,"Ngựa, lừa, la sống/- Loại khác"
4,01022100,Động vật sống họ trâu bò/- Gia súc:/- - Loại t...
5,01022911,Động vật sống họ trâu bò/- Gia súc:/- - Loại k...
6,01022919,Động vật sống họ trâu bò/- Gia súc:/- - Loại k...
7,01022990,Động vật sống họ trâu bò/- Gia súc:/- - Loại k...
8,01023100,Động vật sống họ trâu bò/- Trâu:/- - Loại thuầ...
9,01023900,Động vật sống họ trâu bò/- Trâu:/- - Loại khác


In [19]:
def group_by_hs_prefix(df):
    grouped = defaultdict(list)
    for _, row in df.iterrows():
        mahs_value = row['mahs']
        prefix = str(mahs_value)[:4]  # Chuyển sang chuỗi và lấy 4 ký tự đầu
        grouped[prefix].append(row.to_dict())  # Chuyển hàng thành từ điển
    return grouped

In [20]:
class SimpleHSCodeItem(BaseModel):
    mahs: str
    mo_ta: str

class SimpleHSCodeGroup(BaseModel):
    items: List[SimpleHSCodeItem]

In [21]:
grouped_data = group_by_hs_prefix(data)

In [22]:
grouped_data

defaultdict(list,
            {'0101': [{'mahs': '01012900',
               'mo_ta': 'Ngựa, lừa, la sống/- Ngựa:/- - Loại khác'},
              {'mahs': '01013010',
               'mo_ta': 'Ngựa, lừa, la sống/- Lừa:/- - Loại thuần chủng để nhân giống'},
              {'mahs': '01013090',
               'mo_ta': 'Ngựa, lừa, la sống/- Lừa:/- - Loại khác'},
              {'mahs': '01019000', 'mo_ta': 'Ngựa, lừa, la sống/- Loại khác'}],
             '0102': [{'mahs': '01022100',
               'mo_ta': 'Động vật sống họ trâu bò/- Gia súc:/- - Loại thuần chủng để nhân giống'},
              {'mahs': '01022911',
               'mo_ta': 'Động vật sống họ trâu bò/- Gia súc:/- - Loại khác:/- - - Gia súc đực:/- - - - Bò thiến (SEN)'},
              {'mahs': '01022919',
               'mo_ta': 'Động vật sống họ trâu bò/- Gia súc:/- - Loại khác:/- - - Gia súc đực:/- - - - Loại khác'},
              {'mahs': '01022990',
               'mo_ta': 'Động vật sống họ trâu bò/- Gia súc:/- - Loại khác:/- -

In [23]:
print(type(grouped_data))

<class 'collections.defaultdict'>


In [24]:
def build_prompt_for_description(prefix: str, items: List[Dict]) -> str:
    prompt = """Bạn là chuyên gia về phân loại HS code. Dưới đây là thông tin về các mã HS code thuộc nhóm {prefix}:

{items_list}

Nhiệm vụ: Tái cấu trúc lại mô tả (`mo_ta`) cho từng mã HS code để mô tả rõ ràng, sinh động, và phân biệt các mã trong cùng nhóm. 

Yêu cầu:
- Mô tả mới phải dựa hoàn toàn trên thông tin trong `mo_ta` hiện tại, không thêm hoặc suy diễn thông tin ngoài dữ liệu cung cấp.
- Mô tả cần rõ ràng, tự nhiên, như cách một chuyên gia giải thích cho người dùng thông thường.
- Làm nổi bật sự khác biệt giữa các mã trong nhóm {prefix}, ví dụ: loại động vật, mục đích sử dụng, hoặc đặc điểm cụ thể.
- Mỗi mô tả bắt đầu bằng mã HS (`mahs`), nêu rõ nó thuộc nhóm {prefix}, và giải thích chi tiết nội dung của `mo_ta`.
- Trả về đúng định dạng JSON sau, giữ nguyên cấu trúc đầu vào:

{{
  "{prefix}": [
    {{
      "mahs": "mã HS",
      "mo_ta": "mô tả mới"
    }},
    ...
  ]
}}

Ví dụ:
Nếu đầu vào là:
- 01012900: Ngựa, lừa, la sống/- Ngựa:/- - Loại khác
- 01013010: Ngựa, lừa, la sống/- Lừa:/- - Loại thuần chủng để nhân giống

Đầu ra mong muốn:
{{
  "0101": [
    {{
      "mahs": "01012900",
      "mo_ta": "Mã 01012900 thuộc nhóm 0101 (Ngựa, lừa, la sống), áp dụng cho ngựa, cụ thể là các loại ngựa không thuộc diện thuần chủng hoặc nhân giống."
    }},
    {{
      "mahs": "01013010",
      "mo_ta": "Mã 01013010 thuộc nhóm 0101 (Ngựa, lừa, la sống), áp dụng cho lừa, cụ thể là loại lừa thuần chủng được nuôi để nhân giống."
    }}
  ]
}}
"""
    items_list = "\n".join(f"- {item['mahs']}: {item['mo_ta']}" for item in items)
    return prompt.format(prefix=prefix, items_list=items_list)

In [30]:
import json
import csv

async def fetch_and_save_hscode_csv(output_file="/home/vinh/HS Code/Data/new_mota.csv", model="gemini-2.0-flash-001"):
    all_rows = []  # chứa tất cả các dòng dữ liệu từ mọi prefix
    client = genai.Client(api_key=api_key)

    for prefix, items in grouped_data.items():
        prompt = build_prompt_for_description(prefix, items)
        
        response = client.models.generate_content(
            model=model,
            contents=prompt,
            config={
                "response_mime_type": "application/json",
                "response_schema": SimpleHSCodeGroup
            },
        )

        raw_text = response.text

        try:
            data = json.loads(raw_text)
            if "items" in data and isinstance(data["items"], list):
                all_rows.extend(data["items"])
            else:
                print(f"⚠️ Không tìm thấy 'items' hoặc không phải list tại prefix {prefix}")
        except json.JSONDecodeError:
            print(f"❌ JSON lỗi ở prefix {prefix}: {raw_text}")

    # Ghi dữ liệu ra file CSV
    with open(output_file, mode="w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["mahs", "mo_ta"])
        writer.writeheader()
        writer.writerows(all_rows)

    print(f"✅ Đã lưu kết quả vào file: {output_file}")


In [31]:
output_path = "/home/vinh/HS Code/Data/new_mota.csv"
model = "gemini-2.0-flash-001"
result = await fetch_and_save_hscode_csv(output_path, model)
print(result)


✅ Đã lưu kết quả vào file: /home/vinh/HS Code/Data/new_mota.csv
None


In [32]:
import json

# Giả sử result là dict của bạn
with open("/home/vinh/HS Code/Data/new_mota.csv", "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=4)
